In [8]:
!pip install fastapi
!pip install uvicorn
!pip install joblib
!pip install pandas
!pip install xgboost
!pip install pydantic
!pip install scikit-learn

In [9]:
from google.colab import files
import os

# Upload your trained ETA model (e.g., 'eta_model.pkl')
uploaded = files.upload()
print("Uploaded files:", list(uploaded.keys()))

Saving Rapido2_0.ipynb to Rapido2_0 (1).ipynb
Uploaded files: ['Rapido2_0 (1).ipynb']


In [10]:
%%writefile schemas.py
from pydantic import BaseModel

class RideRequest(BaseModel):
    distance: float        # in km
    hour: int             # 0-23
    day_of_week: int      # 0=Monday, 6=Sunday
    is_peak: int          # 0 or 1

Overwriting schemas.py


In [11]:
%%writefile main.py
from fastapi import FastAPI, HTTPException
from schemas import RideRequest
import joblib
import pandas as pd
import numpy as np

app = FastAPI(title="Rapido ETA Predictor")

# Load model (update filename if different)
try:
    model = joblib.load("eta_model.pkl")
except FileNotFoundError:
    raise RuntimeError("Model file not found. Upload 'eta_model.pkl' first!")

@app.post("/predict-eta")
async def predict_eta(ride: RideRequest):
    try:
        # Prepare input DataFrame
        input_data = pd.DataFrame([[
            ride.distance, ride.hour, ride.day_of_week, ride.is_peak
        ]], columns=['distance', 'hour', 'day_of_week', 'is_peak'])

        # Predict
        eta_minutes = model.predict(input_data)[0]
        return {
            "eta_minutes": float(eta_minutes),
            "confidence": "high"  # Optional: Add model confidence logic
        }
    except Exception as e:
        raise HTTPException(status_code=400, detail=str(e))

@app.get("/")
async def health_check():
    return {"status": "ETA Prediction API is healthy"}

Overwriting main.py


In [12]:
!uvicorn main:app --reload --host 0.0.0.0

INFO:     Will watch for changes in these directories: ['/content']
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
INFO:     Started reloader process [7685] using StatReload
Process SpawnProcess-1:
Traceback (most recent call last):
  File "/content/main.py", line 11, in <module>
    model = joblib.load("eta_model.pkl")
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/joblib/numpy_pickle.py", line 735, in load
    with open(filename, "rb") as f:
         ^^^^^^^^^^^^^^^^^^^^
FileNotFoundError: [Errno 2] No such file or directory: 'eta_model.pkl'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/lib/python3.11/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.11/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/_subpr

In [13]:
{
  "distance": 8.5,
  "hour": 17,
  "day_of_week": 3,
  "is_peak": 1
}

{'distance': 8.5, 'hour': 17, 'day_of_week': 3, 'is_peak': 1}

In [14]:
%%writefile requirements.txt
fastapi==0.95.2
uvicorn==0.22.0
joblib==1.2.0
pandas==2.0.1
xgboost==1.7.5
scikit-learn==1.2.2

Overwriting requirements.txt


In [15]:
%%writefile Dockerfile
FROM python:3.9-slim
WORKDIR /app
COPY . .
RUN pip install --no-cache-dir -r requirements.txt
CMD ["uvicorn", "main:app", "--host", "0.0.0.0", "--port", "8000"]

Writing Dockerfile


In [16]:
!gcloud builds submit --tag gcr.io/YOUR_PROJECT_ID/rapido-eta-api
!gcloud run deploy rapido-eta-api \
  --image gcr.io/YOUR_PROJECT_ID/rapido-eta-api \
  --platform managed \
  --region us-central1 \
  --allow-unauthenticated

ERROR: (gcloud.builds.submit) The required property [project] is not currently set.
It can be set on a per-command basis by re-running your command with the [--project] flag.

You may set it for your current workspace by running:

  $ gcloud config set project VALUE

or it can be set temporarily by the environment variable [CLOUDSDK_CORE_PROJECT]
ERROR: (gcloud.run.deploy) The [project] resource is not properly specified. Please specify the argument [--project] on the command line or set the property [core/project].


In [18]:
!curl -X POST "http://localhost:8000/predict-eta-with-explain" \
  -H "Content-Type: application/json" \
  -d '{"distance":8.5, "hour":17, "day_of_week":3, "is_peak":1}'

curl: (7) Failed to connect to localhost port 8000 after 0 ms: Connection refused
